In [1]:
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Add, Input, Multiply, Concatenate

import os, glob, random
from PIL import Image
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 10
num_classes = 2
epochs = 50
data_augmentation = True
num_predictions = 20
#save_dir = os.path.join(os.getcwd(), 'saved_models')
model_path = 'model0.h5'

img_width = 128
img_height = 128

train_imgs = './datasets/Train/'
test_imgs = './datasets/Test/'

class_label = ['class1', 'class2']

In [3]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Add, Input, Multiply, Concatenate

def scam_face_discriminator():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [4]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

def train():
    print('\ntrain start\n')

    ## define model architecture
    model = scam_face_discriminator()

    ## visualize model
    model.summary()

    for layer in model.layers:
        layer.trainable = True

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='max')
    check_point = keras.callbacks.ModelCheckpoint(
        filepath = os.path.join('model{epoch:02d}-vacc{val_acc:.2f}.hdf5'), 
        monitor='val_acc', verbose=0, save_best_only=True, mode='max')

    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        featurewise_center=True,
        featurewise_std_normalization=True,
        zca_whitening=True
    )

    test_datagen = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        zca_whitening=True
    )

    train_generator = train_datagen.flow_from_directory(
        train_imgs,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        test_imgs,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical')

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=50,
        callbacks=[check_point]
    )

    # Save model and weights
    #if not os.path.isdir(save_dir):
    #    os.makedirs(save_dir)
    #model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('\nSaved trained model at --> %s ' % model_path)
    print('\ntrain end\n')

In [5]:
train()


train start

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        18496  

/home/inouelab/.conda/envs/keras35/lib/python3.5/site-packages/keras/preprocessing/image.py:536: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/inouelab/.conda/envs/keras35/lib/python3.5/site-packages/keras/preprocessing/image.py:544: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/inouelab/.conda/envs/keras35/lib/python3.5/site-packages/keras/preprocessing/image.py:554: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


120/50 [========================================================================] - 52s 434ms/step - loss: 2.9541e-04 - acc: 0.9999 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/50
120/50 [========================================================================] - 51s 425ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/50
120/50 [========================================================================] - 51s 425ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/50
120/50 [========================================================================] - 51s 426ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/50
120/50 [========================================================================] - 51s 424ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/50
120/50 [========================================================================

120/50 [========================================================================] - 51s 428ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 46/50
120/50 [========================================================================] - 51s 428ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 47/50
120/50 [========================================================================] - 51s 428ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 48/50
120/50 [========================================================================] - 51s 428ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 49/50
120/50 [========================================================================] - 51s 428ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 50/50
120/50 [===================================================================

In [ ]:
#test
model.fit_generator(
    generator=train_datagen.flow_from_directory(train_imgs, class_label),
    steps_per_epoch=int(np.ceil(len(list(train_dir.iterdir())) / batchsize)),
    epochs=args.epoch,
    verbose=1,
    validation_data=test_datagen.flow_from_directory(test_dir, classes_label),
    validation_steps=int(np.ceil(len(list(test_dir.iterdir())) / batchsize)))

In [13]:
def load_images_all(dataset_path, shuffle=False):
    print('\tload images <-- {}'.format(dataset_path))
    if not os.path.exists(dataset_path):
        raise Exception('{} not exists'.format(dataset_path))

    cls_dirs = os.listdir(dataset_path)
    cls = 0
    imgs = []
    labels = []
    filepaths = []

    for cls_dir in cls_dirs:
        if not os.path.isdir(dataset_path + '/' + cls_dir): continue
        _imgs, _labels, _filepaths = load_images(dataset_path + '/' + cls_dir, cls)
        imgs += _imgs
        labels += _labels
        filepaths += _filepaths
        cls += 1

    imgs = np.array(imgs)
    labels = np.array(labels)
    filepaths = np.array(filepaths)

    if shuffle:
        s = np.arange(imgs.shape[0])
        np.random.shuffle(s)
        imgs = imgs[s]
        labels = labels[s]
        filepaths = filepaths[s]
    print('\tloaded images\n')
    return imgs, labels, filepaths

def load_images(dataset_path, label, shuffle=False):
    filepaths_jpg = glob.glob(dataset_path + '/*.jp*g')
    filepaths_png = glob.glob(dataset_path + '/*.png')
    filepaths = filepaths_jpg + filepaths_png
    filepaths.sort()
    datasets = []
    labels = []

    for filepath in filepaths:
        img = Image.open(filepath).convert('RGB') ## Gray->L, RGB->RGB
        img = img.resize((img_width, img_height))
        #label = int(filepath.split('/')[-1].split('_')[0])

        x = np.array(img, dtype=np.float32)
        x = x / 255.
        #x = x.reshape(3, INPUT_HEIGHT, INPUT_WIDTH)
        #t = np.array(label, dtype=np.int32)
        t = label
        datasets.append(x)
        labels.append(t)
    if shuffle: random.shuffle(datasets)

    return datasets, labels, filepaths

In [14]:
def test():
    print('\ntest start\n')
    model = scam_face_discriminator()
    #model.summary()
    model.load_weights(model_path)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

    x_test, y_test, paths = load_images_all(test_imgs)
    #y_test = keras.utils.to_categorical(y_test, num_classes)

    # Evaluatemodel with test data set and share sample prediction results
    #evaluation = model.evaluate_generator((x_test, y_test),
    #                                  steps=x_test.shape[0] // batch_size)

    count = 0
    total = x_test.shape[0]
    print('---------------------------------------------------------------------')
    print('evaluate start')
    print('---------------------------------------------------------------------')
    for index, x in enumerate(x_test):
        x = x[None, ...]
        pred = model.predict(x, batch_size=1, verbose=0)
        score = np.max(pred)
        pred_label = np.argmax(pred)

        gt = y_test[index]

        if pred_label == gt: count += 1

        print(' {} : gt {} / pred {}'.format(paths[index], gt, pred_label))

    print('--------------------------------------------------------------------')
    print('accuracy {}  ({} / {})'.format(1.0*count/total, count, total))

    print('\ntest end\n')

In [15]:
test()


test start

	load images <-- ./datasets/Test/
	loaded images

---------------------------------------------------------------------
evaluate start
---------------------------------------------------------------------
 ./datasets/Test//class0/test0.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test1.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test10.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test100.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test101.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test102.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test103.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test104.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test105.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test106.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test107.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test108.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test109.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test11.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test110.jpg : gt 0 / 

 ./datasets/Test//class0/test238.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test239.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test24.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test240.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test241.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test242.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test243.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test244.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test245.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test246.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test247.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test248.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test249.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test25.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test250.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test251.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test252.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test253.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test254.jpg : gt 0 / pr

 ./datasets/Test//class0/test387.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test388.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test389.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test39.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test390.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test391.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test392.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test393.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test394.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test395.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test396.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test397.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test398.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test399.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test4.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test40.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test400.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test401.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test402.jpg : gt 0 / pred

 ./datasets/Test//class0/test535.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test536.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test537.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test538.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test539.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test54.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test540.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test541.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test542.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test543.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test544.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test545.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test546.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test547.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test548.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test549.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test55.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test550.jpg : gt 0 / pred 0
 ./datasets/Test//class0/test551.jpg : gt 0 / pr